In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch

#파이토치 버전 확인, cuda device properties 확인
print('torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

torch 1.13.0+cu116 CPU


In [ ]:
%cd '/content/drive/Othercomputers/Mymac/new/'

/content/drive/Othercomputers/Mymac/new


In [ ]:
# json 파일을 yaml 파일로
import yaml
import json
from glob import glob

%cd /content/drive/Othercomputers/Mymac/new/json/
json_list = glob('*.json')
print(json_list)
for i in range(len(json_list)):
  with open(json_list[i], 'r') as file:
    config = json.load(file)
  file_name = json_list[i].split('.')[0]
  with open('/content/drive/Othercomputers/Mymac/new/yaml/' + file_name + '.yaml', 'w') as yaml_file:
    yaml.dump(config, yaml_file)
  # with open('/content/drive/MyDrive/pre_dataset/yaml/' + file_name + '.yaml', 'r') as yaml_file:
  #   print(yaml_file.read())

/content/drive/Othercomputers/Mymac/new/json
['Short_stature__out__Market__00019.json', 'Short_stature__out__Residential_area__00019.json', 'Short_stature__out__Market__00059.json', 'Short_stature__out__Market__00049.json', 'Short_stature__out__Building_area__00009.json', 'Short_stature__out__Market__00039.json', 'Short_stature__out__Residential_area__00009.json', 'Short_stature__out__Building_area__00019.json', 'Short_stature__out__Market__00009.json', 'Short_stature__out__Market__00029.json']


In [ ]:
# x_center, y_center 구하는 함수
def tocenter(x,y,w,h):
  x1=x+w
  y1=y+h
  cx = (x+x1)/2
  cy = (y+y1)/2

  # normalize
  nx = cx/1280
  ny = cy/720
  nw = w/1280
  nh = h/720

  return nx, ny, nw, nh

In [ ]:
%cd /content/drive/Othercomputers/Mymac/new/yaml/
yaml_list = glob('*.yaml')
print(len(yaml_list))

/content/drive/Othercomputers/Mymac/new/yaml
10


In [ ]:
# yaml 파일에서 필요한 데이터만 추출 - x, y, w, h, 클래스넘버, 클래스이름
# txt 파일 구성: class number, x_center, y_center, width, height (normalized)
import yaml
import os

%cd /content/drive/Othercomputers/Mymac/new/yaml/
yaml_list = glob('*.yaml')
class_name = []
index_list = []
for yaml_name in yaml_list:
    with open(yaml_name, 'r') as file:
      yaml_data = yaml.safe_load(file)
      annotation = yaml_data['annotation']
      ans = annotation['annotations'] # 각 annotations가 저장되어 있는 리스트
      front = yaml_name.split('.')[0]

      #print(len(ans))
      for a in range(len(ans)):
        boxs = ans[a].get('box') # 하나의 annotation에서 box 정보만 추출 (리스트 저장) -> 즉 이미지 하나에 있는 box들
        box_info = []
        scene_name = ans[a].get('atchOrgFileName').split('.')[0]
        if '__' in scene_name:
          txt_name = scene_name
        else:
          txt_name = front + '__' + scene_name



        for b in range(len(boxs)): # 한 개 box의 정보
          box = boxs[b]
          #print(box)
          # 클래스 이름 리스트
          cname = box.get('category_name')
          if cname not in class_name:
            class_name.append(cname)

          # x, y, w, h 각각 리스트로
          box_data = box.get('box')
          #print(box_data)
          x, y, w, h = box_data.get('x'), box_data.get('y'), box_data.get('w'), box_data.get('h')
          # x_center, y_center 함수 불러오기 -> 리스트 저장
          nx, ny, nw, nh = tocenter(x,y,w,h)
          box_coordinate = [class_name.index(cname), nx, ny, nw, nh]
          box_info.append(box_coordinate)

        if len(box_info)==0:
          index_list.append(txt_name)

        # txt 파일에 저장
        f = open('/content/drive/Othercomputers/Mymac/new/labels/' + txt_name + '.txt', 'a')
        for i in box_info:
          value = ' '.join(list(map(str, i)))
          f.write(value +'\n')
        f.close()

/content/drive/Othercomputers/Mymac/new/yaml


In [ ]:
%cd /content/drive/Othercomputers/Mymac/new/images/

img_list = glob('*.png')
len(img_list)

/content/drive/Othercomputers/Mymac/new/images


598

In [ ]:
print(index_list)

['Short_stature__out__Market__00019__scene00414', 'Short_stature__out__Market__00019__scene00827', 'Short_stature__out__Market__00019__scene01299', 'Short_stature__out__Market__00019__scene01653', 'Short_stature__out__Market__00019__scene01771', 'Short_stature__out__Market__00019__scene01830', 'Short_stature__out__Residential_area__00019__scene00001', 'Short_stature__out__Residential_area__00019__scene00060', 'Short_stature__out__Market__00049__scene01535', 'Short_stature__out__Market__00049__scene01712', 'Short_stature__out__Market__00049__scene01771', 'Short_stature__out__Market__00049__scene01830', 'Short_stature__out__Market__00049__scene02302', 'Short_stature__out__Market__00039__scene02007', 'Short_stature__out__Residential_area__00009__scene00296', 'Short_stature__out__Residential_area__00009__scene01948', 'Short_stature__out__Residential_area__00009__scene02007', 'Short_stature__out__Residential_area__00009__scene02066']


In [ ]:
len(index_list)

18

In [ ]:
for idx in index_list:
  os.remove('/content/drive/Othercomputers/Mymac/new/images/' + idx + '.png')
  os.remove('/content/drive/Othercomputers/Mymac/new/labels/' + idx + '.txt')

In [ ]:
%cd /content/drive/Othercomputers/Mymac/new/images/

img_list = glob('*.png')
len(img_list)

/content/drive/Othercomputers/Mymac/new/images


580

In [ ]:
%cd /content/drive/Othercomputers/Mymac/new/labels/

txt_list = glob('*.txt')
len(txt_list)

/content/drive/Othercomputers/Mymac/new/labels


580

In [ ]:
# 사진 개수와 레이블 개수 다른거 확인.. 찾아서 삭제
c_img_list = sorted(img_list)
t_img_list = sorted(txt_list)

for i in range(len(c_img_list)):
  if c_img_list[i].split('.')[0] != t_img_list[i].split('.')[0]:
    print(c_img_list[i])
    break

Short_stature__out__Residential_area__00007__scene01299 (1).png


In [ ]:
os.remove('/content/drive/MyDrive/pre_dataset/images/Short_stature__out__Residential_area__00007__scene01299 (1).png')

In [ ]:
from glob import glob


img_list = glob('/content/dataset/images/*.png')

print(len(img_list))

0


In [ ]:
# 리스트를 txt파일로 생성
with open('/content/dataset/train.txt', 'w') as f:
	f.write('\n'.join(train_img_list) + '\n')
    
with open('/content/dataset/val.txt', 'w') as f:
	f.write('\n'.join(val_img_list) + '\n')

In [ ]:
# yaml 파일에 필요한 정보: 학습데이터 경로, 클래스 개수, 클래스 이름
import yaml
%cd /content/dataset/
with open('data.yaml', 'r') as f:
  data = yaml.safe_load(f)

print(data)

data['train'] = '/content/dataset/train.txt'
data['val'] = '/content/dataset/val.txt'
data['nc'] = len(class_name)
data['names'] = class_name

with open('/content/dataset/data.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

/content/dataset
{'names': ['car', 'pot', 'bicycle', 'person', 'powerpole', 'tree', 'signboard', 'bollard', 'motorcycle', 'colorcone'], 'nc': 10, 'train': '/content/dataset/train.txt', 'val': '/content/dataset/val.txt'}
{'names': ['car', 'pot', 'bicycle', 'person', 'powerpole', 'tree', 'signboard', 'bollard', 'motorcycle', 'colorcone'], 'nc': 10, 'train': '/content/dataset/train.txt', 'val': '/content/dataset/val.txt'}
